You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# Build a Crew to Tailor Job Applications

If you are running this notebook on your own machine, you may need to install the required libraries:

- `crewai`: A library for creating and managing AI agents and tasks.
- `langchain`: A library for building applications with language models.
- `langchain_community`: Community-contributed extensions for LangChain.
- `openai`: OpenAI's API client library.
- `boto3`: The Amazon Web Services (AWS) SDK for Python.
- `python-docx`: A library for creating, modifying, and extracting information from Microsoft Word documents.
- `langchain_aws`: AWS-specific extensions for LangChain.
- `crewai_tools`: Tools for enhancing the functionality of CrewAI.
- `anthropic`: A library for interacting with Anthropic's language models.
- `litellm`: A lightweight library for interacting with various language models.


In [ ]:
%pip install crewai 

In [ ]:
%pip install docx2txt

In [ ]:
%pip install --upgrade langchain langchain-community openai boto3

In [ ]:
%pip install python-docx

In [ ]:
%pip install langchain_aws

In [ ]:
%pip install crewai_tools 
%pip install langchain_community

In [ ]:
%pip install anthropic litellm langchain_community

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew, LLM

In [3]:
import litellm
from litellm import AnthropicChatCompletion

- As an LLM for your agents, you'll be using Haiku via Bedrock. This setup leverages the Bedrock platform to access the Haiku model, which will be used by the agents to perform their tasks effectively.

In [ ]:
# Setting up the Serper API Key
import sys
import os
from utils import get_serper_api_key, pretty_print_result

# Retrieve the API keys
serper_api_key = get_serper_api_key()

# Set the environment variables
os.environ["SERPER_API_KEY"] = serper_api_key

# Print the environment variables to validate
# print("SERPER_API_KEY:", os.environ["SERPER_API_KEY"])

# Assuming your 'resume' folder is within the current working directory
file_path = '/resume/sample_resume.docx'

In [7]:
## Initialize a language model using the Amazon Bedrock. You can experiment with different models by changing the model parameter.

from langchain_community.llms import Bedrock
 
llm = LLM(model="bedrock/anthropic.claude-3-5-haiku-20241022-v1:0", temperature=0.7)

In [8]:
import crewai_tools

In [9]:
from crewai_tools import DOCXSearchTool

# CrewAI Tools

In [ ]:
# This code initializes a semantic search tool using Amazon Bedrock as the LLM provider and embedder.
semantic_search_resume = DOCXSearchTool(
    docx=file_path,
    config=dict(
        llm=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock as the LLM provider
            config=dict(
                model="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Specify the Bedrock-supported model
                # temperature=0.5,
                # top_p=1,
                # stream=True,
            ),
        ),
        embedder=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock for embeddings
            config=dict(
                model="amazon.titan-embed-text-v1",  # Bedrock-supported embedding model
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [11]:
# Import necessary tools from crewai_tools module and initialize them
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FileReadTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path)

In [ ]:
# This code reads a .docx file and displays its content as a Markdown preview in a Jupyter notebook.

from docx import Document

def display_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])  # Extract text from all paragraphs
    return text

file_path = "resume/sample_resume.docx"  # Upload your resume or use a sample 
resume_text = display_docx(file_path)

from IPython.display import display, Markdown
display(Markdown(f"### Resume Preview\n\n{resume_text}"))

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [13]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [14]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [15]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [16]:
# Agent 4: Interview Coach
interview_preparer = Agent(
    role="Engineering Interview Coach",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [17]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [18]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal website "
        "({personal_website}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [19]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [20]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.

In [23]:
## USE THIS FOR BEDROCK
from langchain_community.llms import Bedrock
 
manager_llm = LLM(model="anthropic.claude-3-5-sonnet-20241022-v2:0", temperature=0.7)

In [24]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    llm=manager_llm,
    verbose=True,
    debug=True
)

2025-02-01 19:57:42,964 - 8571142720 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [25]:
job_application_inputs = {
    'job_posting_url': 'https://amazon.jobs/en/jobs/2821385/worldwide-specialist-solutions-architect-genai', # Replace the job posting URL
    'github_url': 'https://github.com/viktoriasemaan', # Replace with your GitHub URL
    'personal_website': 'your_personal_website_url' # Replace with your personal website URL
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)